# Image Endpoint Tester

Manually test CV HTTP endpoint by uploading images and displaying the response

In [1]:
import os

#os.environ['create_image']='True'
#os.environ['repository']='romeokienzler'
#os.environ['version']='0.14'
#
#os.environ['install_requirements']='True'

In [2]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM registry.access.redhat.com/ubi8/python-39
    RUN pip install ipython nbformat flask
    ADD image-endpoint.ipynb /
    ENTRYPOINT ["ipython","/image-endpoint.ipynb","> /tmp/component.log","2> /tmp/component.err"]
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t claimed-predict-image-endpoint:`echo $version` .
    !docker tag claimed-predict-image-endpoint:`echo $version` `echo $repository`/claimed-predict-image-endpoint:`echo $version`
    !docker push `echo $repository`/claimed-predict-image-endpoint:`echo $version`
elif bool(os.environ.get('install_requirements',False)):
    !pip install voila

In [3]:
import ipywidgets as widgets
import requests
from ipywidgets import VBox, HBox, Button, Image, FileUpload, Text, IntProgress

In [4]:
# menue

application_button = Button(
        description="Application",
        disabled=True
)

config_button = Button(
        description="Config",
        disabled=False
)




menue = VBox((
    application_button,
    config_button
))


In [5]:
# application


uploader = FileUpload(
    accept='.png',
    multiple=False
)



image = Image()
image.layout.display = 'none'



progress = IntProgress(
    value=0,
    min=0,
    max=2,
    description='Loading:',
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)

def report_change(change):
    new_value = change.new
    if type(new_value) is dict and len(new_value) > 0:
        first_key = next(iter(new_value))
        if '.png' in first_key:
            image_data = new_value[first_key]['content']

            files= {'file': ('image.png',image_data,'multipart/form-data',{'Expires': '0'}) }
            with requests.Session() as s:
                progress.value = 1
                response = s.post('http://127.0.0.1:8080/process-image',files=files)
                progress.value = 2
                print(response.status_code)
                image.value = response.content
                image.layout.display = 'block'

    return

uploader.observe(report_change)


application = VBox((uploader,progress, image))

In [6]:
# config
endpoint_url_textfield = Text(
    value='http://127.0.0.1:8080/process-image',
    description='Endpoint:',
    disabled=False
)

config = endpoint_url_textfield
config.layout.display = 'none'





In [7]:
# button callback handlers

def application_button_clicked(m):
    application_button.disabled = True
    config_button.disabled = False
    config.layout.display = 'none'
    application.layout.display = 'block'




application_button.on_click(application_button_clicked)

def config_button_clicked(m):
    application_button.disabled = False
    config_button.disabled = True
    config.layout.display = 'block'
    application.layout.display = 'none'




config_button.on_click(config_button_clicked)

In [8]:
display(HBox((menue,VBox((application, config)))))
